In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import pylab

Загружаем выборки.

In [2]:
data1 = pd.read_excel('Data_Extract_From_Gender_Statistics.xlsx', encoding = 'utf8')
def_data1 = pd.read_excel('Data_Extract_From_Gender_Statistics.xlsx', sheetname=1, encoding = 'utf8')

data2 = pd.read_excel('Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', encoding='utf8')
def_data2 = pd.read_excel('Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', sheetname=1, encoding='utf8')

data3 = pd.read_excel('Data_Extract_From_Millennium_Development_Goals.xlsx', encoding='utf8')
def_data3 = pd.read_excel('Data_Extract_From_Millennium_Development_Goals.xlsx', sheetname=1, encoding='utf8')

data_cnt = pd.read_csv('all.csv', encoding='utf8')

Очищаем данные от больших пропусков, точек и пр.

In [3]:
def Fillnan(data, years_c):
    for name in years_c:
        data[name] = data[name].apply(lambda x: np.nan if x==('..') else float(x))
    return data
    
def Dropempt(data, years_c):
    tmp=[]
    for ind, item in enumerate(data[years_c].as_matrix()):
        if np.nansum(item) == 0:
            tmp.append(ind)
    data = data.drop(data.index[tmp]).reset_index()
    return data.drop('index', axis=1)
    
def Clearing(data):
    years_c = [item for ind, item in enumerate(np.array(data.columns)) if ind not in (range(0,4))]
    if data['Country Code'].get_value(len(data)-5) == np.nan:
        data=data.drop(data.index[[range(len(data)-5, len(data))]]).reset_index()
    data = Dropempt(Fillnan(data, years_c), years_c)
    return data

Сортировка по регионам.

In [4]:
def Make_region(code):
    #print code
    if code in data_cnt['name'].values:
        #print code
        name = (data_cnt['name'][data_cnt['name'] == code].index.tolist())[0]
        return data_cnt['region'].get_value(name)
    else:
        return np.nan

def Sorting(data):
    if data['Country Name'].get_value(len(data)-5) == np.nan:
        data=data.drop(data.index[[range(len(data)-5, len(data))]]).reset_index()
    elif 'Region' not in data.columns:
        data['Region'] = data['Country Name'].apply(lambda x: Make_region(x))
        data['Region'].dropna()
        return data.reset_index()
    else:
        return data

In [5]:
data1 = Clearing(data1)
#заметил много шума.
del data1['2016 [YR2016]']

data2 = Clearing(data2)
del data2['2001 [YR2001]']

data3 = Clearing(data3)
del data3['2016 [YR2016]']

Эвристика
Преимущество данного подхода в том, что мы не потеряем единственные значения у некоторых стран. Но при этом, отразим актуальность, а значит и реальность.

In [6]:
def data_inc(data):
    magic = 0.08
    for ind, item in enumerate(np.array(data.columns)):
        if ind not in (range(0,4)):
        #попробую с интерполяцией
            data[item] = data[item].apply(lambda x: ((ind-2)*magic)*x if ((ind-2)*magic)>1 else x).interpolate()
    return data

def R_mean(data):
    if ('Mean' not in data.columns) and ('Region' not in data.columns):
        data['Mean']= data[[item for ind, item in enumerate(np.array(data.columns)) if ind not in (range(0,4))]].apply(lambda x: np.nanmean(x), axis = 1)
    elif ('Region' in data.columns):
        data['Mean']= data[[item for ind, item in enumerate(np.array(data.columns)) if ind not in [(range(0,4)), len(data.columns)-1]]].apply(lambda x: np.nanmean(x), axis = 1)
    return data

Чувствую себя отвратительно за такой код. Доделаю до конца и поправлю. 

In [7]:
data1 = Sorting(R_mean(data_inc(data1)))
data2 = Sorting(R_mean(data_inc(data2)))
data3 = Sorting(R_mean(data_inc(data3)))

Вот тут буду коррелировать, строить графики и пр

In [26]:
data1['Mean'][data1['Series Code']=='SP.POP.DPND'][data1['Region']=='Asia']

276    99.737395
283    47.776265
287    44.612129
289    35.142952
290    61.056670
296    57.908157
301    42.849394
306    64.097812
315    37.854602
325    44.040880
353    48.618766
375    58.867425
376    52.817248
378    84.159780
380    63.017822
383    56.987059
384    70.052560
385    48.727056
391    36.982187
400    51.883333
413    51.729113
414    56.800894
426    48.849009
430    54.811398
432    74.937535
442    46.469442
445    72.353414
450    65.060092
456    24.401256
462    53.726059
467    38.068137
478    50.306143
489    71.444808
490    69.399828
492    41.566410
493    96.373539
498    53.772610
499    55.394652
502    20.821139
507    56.222398
Name: Mean, dtype: float64

In [17]:
data1['Mean'][data1['Series Code']=='SP.POP.AG00.FE.IN'][data1['Region']=='Asia']

517    4.960037e+05
523    2.012780e+04
527    7.683434e+04
529    8.908243e+03
530    1.602973e+06
536    7.055651e+03
541    3.145094e+03
546    1.738185e+05
553    7.824735e+06
563    6.255023e+03
581    2.849262e+04
596    1.243126e+07
597    2.326190e+06
599    5.055892e+05
601    7.548108e+04
604    5.393666e+05
605    8.645477e+04
606    1.667348e+05
612    2.975486e+04
616    3.292990e+04
626    2.824965e+05
627    3.401746e+03
635    2.984402e+04
639    5.087708e+05
641    3.095095e+05
649    3.199645e+04
650    2.270972e+06
655    1.145614e+06
659    9.242283e+03
665    2.975438e+05
670    2.456779e+04
678    1.749926e+05
686    2.419293e+05
687    1.055337e+05
689    3.994073e+05
690    1.745711e+04
695    6.585628e+05
696    5.328954e+04
699    4.013674e+04
703    2.975501e+05
Name: Mean, dtype: float64

In [30]:
data_cnt['region'].head()

0       Asia
1     Europe
2     Europe
3     Africa
4    Oceania
Name: region, dtype: object

In [33]:
def Correlation(data, def_data):
    rez=pd.Series()
    for region in data_cnt['region']:
        data['Mean'][data['Series Code']=='SP.POP.DPND'][data['Region']==region]

In [46]:
test = pd.Series()
test.columns = data_cnt['region'].dropna().unique()

In [49]:
test.index = data_cnt['region'].dropna().unique()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements